# Data validation and cost estimation

Reference: https://cookbook.openai.com/examples/chat_finetuning_data_prep

In [ ]:
# Import required libraries 
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [4]:
#Load the training dataset 
data_path = "training_data.jsonl"

with open(data_path, 'r', encoding= 'utf-8') as f:
    dataset = [json.loads(line) for line in f] 

print(f"Loaded {len(dataset)} training examples from {data_path}") 

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Loaded 600 training examples from training_data.jsonl
Num examples: 600
First example:
{'role': 'system', 'content': 'You are a highly intelligent emotion classification assistant. You carefully read a comment, and label it with one or more pre-selected emotion labels to it. The emotion labels are listed here: admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarassment, excitement, fear, gratitude, grief, joy, love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, neutral.Your output should be just the emotion label that you are applying to the comment, and if you think there are multple labels then output the labels separated by commas.'}
{'role': 'user', 'content': 'Label the emotion of this comment: You just make her sound awesome.'}
{'role': 'assistant', 'content': 'admiration'}


# Format validation checks

We'll run the following checks on our dataset: 

* __Data Type Check:__ Ensures each example in the dataset is a Python dictionary; flags items that are not dicts.
* __Missing Messages List Check:__ Confirms each example contains a "messages" list; flags examples with a missing or empty messages field.
* __Message Required Keys Check:__ Verifies each message includes the required "role" and "content" keys; flags messages missing either.
* __Unrecognized Message Key Check:__ Ensures each message only contains allowed keys (role, content, name, function_call, weight); flags messages with unexpected fields.
* __Role Validation Check:__ Confirms role values are valid (system, user, assistant, function); flags any unrecognized roles.
* __Content or Function Call Check:__ Validates that each message has either non-empty string content or a function_call; flags missing or incorrectly typed content.
* __Assistant Message Presence Check:__ Ensures each example includes at least one message with role="assistant"; flags examples missing an assistant response.
* __Final Error Summary Check:__ After validation, prints a summary of all detected formatting issues or reports “No errors found” if the dataset is clean.

In [5]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


# Functions to estimate the token counts 

* __Total Tokens (num_tokens_from_messages):__ Returns an approximate total token count for the entire message list i.e., input tokens.
    * Adds a fixed per-message cost i.e., 3 tokens by default
    * Adds tokenized length of every field (content, role, name, etc.); Adds +1 extra token when a "name" field is present.
    * Adds 3 tokens at the end as default for formatting tokens.

* __Assistant Tokens (num_assistant_tokens_from_messages):__ Returns an approximate total token count for output tokens.
    * Looks only at messages where role == "assistant".
    * Counts tokenized length of the assistant’s content. 

We also define a function to calculate some descriptive stats for the dataset and print it out.

In [9]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [10]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 155, 194
mean / median: 169.36, 169.0
p5 / p95: 159.0, 181.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 1, 8
mean / median: 2.0183333333333335, 2.0
p5 / p95: 1.0, 4.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


In [11]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~101616 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~304848 tokens
